In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from hyperopt import fmin, tpe, hp, STATUS_OK
from sklearn.model_selection import cross_val_score
import time

In [2]:
train = pd.read_csv("sets_de_datos/dima_train_with_features_6.csv",
                    dtype={ \
                            'antiguedad' : np.float32, 
                            'habitaciones' : np.float32, 
                            'garages' : np.float32, 
                            'banos' : np.float32,
                            'metroscubiertos' : np.float32, 
                            'metrostotales' : np.float32, 
                            'idzona' : np.float32, 
                            'lat' : np.float32, 
                            'lng' : np.float32, 
                            'gimnasio' : np.float32, 
                            'usosmultiples' : np.float32, 
                            'piscina' : np.float32, 
                            'escuelascercanas' : np.float32,
                            'centroscomercialescercanos' : np.float32, 
                            'precio' : np.float32, 
                            'servicios_cercanos' : np.float32,
                            'utilidades_extra' : np.float32, 
                            'tipodepropiedad_mean_antiguedad' : np.float32,
                            'tipodepropiedad_mean_habitaciones ' : np.float32, 
                            'tipodepropiedad_mean_garages' : np.float32,
                            'tipodepropiedad_mean_banos' : np.float32, 
                            'tipodepropiedad_mean_metroscubiertos' : np.float32,
                            'tipodepropiedad_mean_utilidades_extra' : np.float32,
                            'tipodepropiedad_mean_servicios_cercanos' : np.float32, 
                            'ciudad_mean_antiguedad' : np.float32,
                            'ciudad_mean_habitaciones' : np.float32, 
                            'ciudad_mean_garages' : np.float32, 
                            'ciudad_mean_banos': np.float32,
                            'ciudad_mean_metroscubiertos' : np.float32, 
                            'ciudad_mean_utilidades_extra' : np.float32,
                            'ciudad_mean_servicios_cercanos' : np.float32, 
                            'provincia_mean_antiguedad' : np.float32,
                            'provincia_mean_habitaciones' : np.float32, 
                            'provincia_mean_garages' : np.float32,
                            'provincia_mean_banos' : np.float32, 
                            'provincia_mean_metroscubiertos' : np.float32,
                            'provincia_mean_utilidades_extra' : np.float32, 
                            'provincia_mean_servicios_cercanos' : np.float32,
                            'ciudad_mean_lat' : np.float32, 
                            'ciudad_mean_lng' : np.float32, 
                            'provincia_mean_lat' : np.float32,
                            'provincia_mean_lng' : np.float32, 
                            'coseno_distancia_angular_al_centro_ciudad' : np.float32,
                            'coseno_distancia_angular_al_centro_provincia' : np.float32,
                            'coseno_distancia_angular_al_centro_pais' : np.float32, 
                            'año' : np.float32, 
                            'mes' : np.float32, 
                            'dia' : np.float32,
                            'metros_no_cubiertos' : np.float32, 
                            'cantidad_espacios' : np.float32, 
                            'metros_x_espacio' : np.float32,
                            'metros_x_habitaciones' : np.float32, 
                            'metros_x_garages' : np.float32, 
                            'metros_x_banos' : np.float32,
                            'metros_x_utilidades_extra' : np.float32, 
                            'ratio_metros_cubiertos' : np.float32,
                            'distancia_minima_Terreno_comercial' : np.float32,
                            'distancia_minima_Local_Comercial' : np.float32,
                            'distancia_minima_Oficina_comercial' : np.float32,
                            'distancia_minima_Local_en_centro_comercial' : np.float32,
                            'distancia_minima_Bodega_comercial' : np.float32, 
                            'banos_preciopromedio_ciudad' : np.float32,
                            'habitaciones_preciopromedio_ciudad' : np.float32, 
                            'garages_preciopromedio_ciudad' : np.float32,
                            'banos_preciopromedio_metroscubiertos' : np.float32,
                            'habitaciones_preciopromedio_metroscubiertos' : np.float32,
                            'garages_preciopromedio_metroscubiertos' : np.float32, 
                            'precio_x_m2' : np.float32,
                            'tipodepropiedad_mean_precio' : np.float32, 
                            'titulo_cantidad_palabras_importantes' : np.float32,
                            'descripcion_cantidad_palabras_importantes' : np.float32,
                            'direccion_cantidad_palabras_importantes' : np.float32,
                            'titulo_cantidad_caracteres_en_palabras_importantes' : np.float32,
                            'descripcion_cantidad_caracteres_en_palabras_importantes' : np.float32,
                            'direccion_cantidad_caracteres_en_palabras_importantes' : np.float32,
                            'titulo_longitud_media_de_palabra' : np.float32,
                            'descripcion_longitud_media_de_palabra' : np.float32,
                            'direccion_longitud_media_de_palabra' : np.float32, 
                            'titulo_cantidad_stopwords' : np.float32,
                            'descripcion_cantidad_stopwords' : np.float32, 
                            'direccion_cantidad_stopwords' : np.float32,
                            'titulo_cantidad_signos_puntacion' : np.float32,
                            'descripcion_cantidad_signos_puntacion' : np.float32,
                            'direccion_cantidad_signos_puntacion' : np.float32,
                            'direccion_cantidad_palabras_en_mayuscula' : np.float32,
                            'direccion_cantidad_titulos' : np.float32, 
                            'titulo_cantidad_palabras_top_k' : np.float32,
                            'descripcion_cantidad_palabras_top_k' : np.float32,
                            'direccion_cantidad_palabras_top_k' : np.float32,
                            'titulo_cantidad_palabras_bottom_k' : np.float32,
                            'descripcion_cantidad_palabras_bottom_k' : np.float32,
                            'direccion_cantidad_palabras_bottom_k' : np.float32,
                            'titulo_cantidad_prefijos_top_k' : np.float32, 
                            'descripcion_cantidad_prefijos_top_k' : np.float32,
                            'direccion_cantidad_prefijos_top_k' : np.float32, 
                            'titulo_cantidad_postfijos_top_k' : np.float32,
                            'descripcion_cantidad_postfijos_top_k' : np.float32,
                            'direccion_cantidad_postfijos_top_k' : np.float32, 
                            '0' : np.float32, 
                            '1' : np.float32, 
                            '2' : np.float32, 
                            '3' : np.float32, 
                            '4' : np.float32, 
                            '5' : np.float32, 
                            '6' : np.float32,
                            '7' : np.float32,
                            'distancia_euclideana_al_origen' : np.float32, 
                            'distancia_minima_comercial' : np.float32,
                            'coseno_maximo_ciudad_pais' : np.float32,
                            'ciudad_mean_antiguedad_sobre_provincia_mean_antiguedad' : np.float32,
                            'tipodepropiead_mean_utilidades_extra_sobre_ciudad_mean_utilidades_extra' : np.float32,
                            'antiguedad_sobre_tipodepropiedad_mean_antiguedad' : np.float32,
                            'direccion_cantidad_al_menos_una_mayuscula' : np.float32,
                            'direccion_cantidad_fijos_top_k' : np.float32, 
                            'titulo_cantidad_fijos_top_k' : np.float32,
                            'titulo_palabras_top_k_sobre_total_palabras' : np.float32,
                            'ciudad_distancia_al_origen':  np.float32, 
                            'ciudad_mean_mean_todas' : np.float32,
                            'ciudad_mean_antiguedad_sobre_mean_metrocubiertos' : np.float32
                            }
                    )

In [3]:
train.set_index('id', inplace = True)

In [4]:
TARGET = 'precio'
FEATURES = ['descripcion_longitud_media_de_palabra', 'metrostotales',
       'metros_x_espacio', 'idzona', 'lat', 'titulo_longitud_media_de_palabra',
       'distancia_minima_Oficina_comercial',
       'distancia_minima_Local_en_centro_comercial',
       'distancia_minima_Bodega_comercial', 'metros_x_garages',
       'metroscubiertos', 'metros_x_habitaciones', 'metros_x_banos',
       'distancia_minima_Terreno_comercial',
       'distancia_minima_Local_Comercial',
       'titulo_cantidad_caracteres_en_palabras_importantes',
       'descripcion_cantidad_palabras_top_k',
       'descripcion_cantidad_caracteres_en_palabras_importantes',
       'coseno_distancia_angular_al_centro_provincia',
       'descripcion_cantidad_stopwords',
       'descripcion_cantidad_signos_puntacion',
       'direccion_longitud_media_de_palabra',
       'titulo_palabras_top_k_sobre_total_palabras',
       'habitaciones_preciopromedio_ciudad', 'antiguedad', 'lng',
       'garages_preciopromedio_ciudad',
       'direccion_cantidad_caracteres_en_palabras_importantes',
       'coseno_distancia_angular_al_centro_ciudad',
       'banos_preciopromedio_ciudad', 'descripcion_cantidad_prefijos_top_k',
       'precio_x_m2', 'año', 'titulo_cantidad_fijos_top_k',
       'descripcion_cantidad_palabras_importantes',
       'descripcion_cantidad_postfijos_top_k', 'ciudad_mean_mean_todas',
       'ciudad_mean_antiguedad_sobre_mean_metrocubiertos',
       'habitaciones_preciopromedio_metroscubiertos', 'cantidad_espacios',
       'direccion_cantidad_al_menos_una_mayuscula',
       'direccion_cantidad_fijos_top_k', 'ciudad_distancia_al_origen', ]

In [5]:
X = train[FEATURES]
y = train[TARGET]

In [6]:
K = 3
def objective(hyper_parametros):

    xgb_regressor = XGBRegressor(objective = 'reg:squarederror',**hyper_parametros)
    
    score = cross_val_score(xgb_regressor, X, y, 
                            scoring='neg_mean_absolute_error', cv=K).mean()
    
    print("SCORE: {:.3f} params {}".format(score, hyper_parametros))
    
    return {'loss': -score, 'status': STATUS_OK}

In [7]:
space = {
    "n_estimators": hp.randint("xgb_re__n_estimators", 200) + 500, 
    "learning_rate": hp.uniform("xgb_re__learning_rate", 0.01, 0.03), 
    "gamma": hp.uniform("xgb_re__gamma", 3, 5),
    "lambda": hp.uniform("xgb_re__lambda", 1, 2),
    "max_depth": hp.randint("xgb_re__max_depth", 4) + 16, 
    "min_child_weight": hp.randint("xgb_re__min_child_weight", 1000) + 8,
    'colsample_bytree': 0.8,
}

In [8]:
N_ITER = 20
rstate = np.random.RandomState(42) 
t0 = time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=N_ITER,
            rstate = rstate)
t1 = time.time()
print('Tiempo = {0:.2f} horas'.format((t1 - t0)/3600))

  0%|                                                                             | 0/20 [00:00<?, ?it/s, best loss: ?]

c:\users\juani\appdata\local\programs\python\python36\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

c:\users\juani\appdata\local\programs\python\python36\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

c:\users\juani\appdata\local\programs\python\python36\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



SCORE: -620828.500 params {'colsample_bytree': 0.8, 'gamma': 3.970374385951927, 'lambda': 1.3194472510397675, 'learning_rate': 0.011381026817472084, 'max_depth': 18, 'min_child_weight': 898, 'n_estimators': 541}
  5%|██▉                                                       | 1/20 [28:36<9:03:32, 1716.43s/it, best loss: 620828.5]

c:\users\juani\appdata\local\programs\python\python36\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

c:\users\juani\appdata\local\programs\python\python36\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

c:\users\juani\appdata\local\programs\python\python36\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



SCORE: -579476.667 params {'colsample_bytree': 0.8, 'gamma': 4.882310564371168, 'lambda': 1.3292628550216525, 'learning_rate': 0.027127441306839584, 'max_depth': 19, 'min_child_weight': 671, 'n_estimators': 674}
 10%|████▋                                          | 2/20 [1:09:56<9:43:36, 1945.38s/it, best loss: 579476.6666666666]

...

SCORE: -546858.875 params {'colsample_bytree': 0.8, 'gamma': 4.33933557109701, 'lambda': 1.1231840463016995, 'learning_rate': 0.029821395947830544, 'max_depth': 18, 'min_child_weight': 44, 'n_estimators': 652}
 90%|██████████████████████████████████████████████▊     | 18/20 [11:23:07<1:20:01, 2400.87s/it, best loss: 546858.875]

c:\users\juani\appdata\local\programs\python\python36\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

c:\users\juani\appdata\local\programs\python\python36\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

c:\users\juani\appdata\local\programs\python\python36\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



SCORE: -600396.604 params {'colsample_bytree': 0.8, 'gamma': 4.20083814772583, 'lambda': 1.9661037229320653, 'learning_rate': 0.017847133548675807, 'max_depth': 18, 'min_child_weight': 860, 'n_estimators': 571}
 95%|███████████████████████████████████████████████████▎  | 19/20 [11:54:40<37:28, 2248.66s/it, best loss: 546858.875]

c:\users\juani\appdata\local\programs\python\python36\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

c:\users\juani\appdata\local\programs\python\python36\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \

c:\users\juani\appdata\local\programs\python\python36\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



SCORE: -578486.250 params {'colsample_bytree': 0.8, 'gamma': 4.206224440970836, 'lambda': 1.6498172284765338, 'learning_rate': 0.025223780101520266, 'max_depth': 16, 'min_child_weight': 485, 'n_estimators': 604}
100%|██████████████████████████████████████████████████████| 20/20 [12:28:27<00:00, 2245.35s/it, best loss: 546858.875]
Tiempo = 12.47 horas


In [10]:
params = {'colsample_bytree': 0.8, 'gamma': 4.33933557109701, 'lambda': 1.1231840463016995, 'learning_rate': 0.029821395947830544, 'max_depth': 18, 'min_child_weight': 44, 'n_estimators': 652}
model = XGBRegressor(**params)